# Specialize For SJVAIR

In [8]:
import pandas as pd
import time
import csv
import requests
import datetime
import json
import os

class GetSjvAir:
    
    def __init__(self , list_sensor , start_date , end_date , data , data_type): 
        self.list_sensor = list_sensor
        self.start_date = start_date
        self.end_date = end_date
        self.data = data
        self.data_type = data_type
        self.api_prupleair = {'X-API-Key': "5E56E968-4C34-11EE-A77F-42010A800009"}
        
    def get_data_json_sjvair(self , name):
        
        response = requests.get(f"https://www.sjvair.com/api/1.0/monitors/{name}")
            
        sensor_data = response.json()
        
        result_list = {}
        
        if "name" in self.data:

            result_list = {
                'sensor_id': name,
                'name': sensor_data['data']['name'],
            }
        
        elif "log" in self.data or "lat" in self.data:
            result_list = {
                'sensor_id': name,
                'log': sensor_data['data']['position']['coordinates'][0],
                'lat': sensor_data['data']['position']['coordinates'][1],
            }
        
        return result_list
        
    def get_data_sjvair(self):
        
        parameters = {'timestamp__gte': self.start_date, 'timestamp__lt': self.end_date}
        
        if not os.path.exists("sjvair_folder"):
            os.makedirs("sjvair_folder")

        for i in self.list_sensor:
            
            all_data = []
            
            csv_response = requests.get(f"https://www.sjvair.com/api/1.0/monitors/{i}/entries/csv/", params=parameters)
            
            with open(f"sjvair_folder/SJVAir_{i}.csv", "wb") as f:
                f.write(csv_response.content)

            data_df = pd.read_csv(f"sjvair_folder/SJVAir_{i}.csv")
            
            result_list = self.get_data_json_sjvair(name=i)
            
            for j in range(len(data_df)):
                
                val = data_df.iloc[j]
                
                val_data = {}
                
                for k in self.data:
                    
                    try:
                        
                        val_data[k] = val[k]
                        
                    except KeyError:
                        
                        continue
                    
                val_data.update(result_list)
                
                all_data.append(val_data)

            df_final = pd.DataFrame(all_data)
            
            df_final.to_csv(f"sjvair_folder/SJVAir_{i}.csv", index=False)
            
            print(f"Finish Download sjvair_folder/SJVAir_{i}.csv")
            
            
            
############################################################################


# Specialize For PurpleAIr

In [6]:
class GetPurpleAirData:
    
    def __init__(self, sensor_list, start_date, end_date, headers, fields):
        self.sensor_list = sensor_list
        self.start_date = start_date
        self.end_date = end_date
        self.headers = headers
        self.fields = fields
        
    def get_data_purpleair(self):
        
        for index in range(len(self.sensor_list)):

            url = f'https://api.purpleair.com/v1/sensors/{self.sensor_list[index]}/history/csv?fields={",".join(self.fields)}'
            
            response = requests.get(url, headers=self.headers)
            
            if not os.path.exists("purpleair_folder"):
                os.makedirs("purpleair_folder")
                
            filename = f"purpleair_folder/sensor_{self.sensor_list[index]}_data.csv"
            
            with open(filename , "wb") as f:
                
                f.write(response.content)
                
            print(f"Finish Download {filename}")

# Main Program

In [10]:

if __name__ == '__main__':
    
    datatypes = input("Choose Which data you want Download (SJVAIR OR PurpleAir)")
    
    if datatypes.upper() == 'SJVAIR':
        
        data = GetSjvAir(list_sensor=["yiz5lVBeRf26NdX-KtrPlg" , "K6rfX7COTLKuQIFIh00gjA"] , start_date = "2023-11-13" , end_date= "2023-11-15" , data = ["name" , "celsius" , "pm25" , "humidity" , 'timestamp'] , data_type="SJVAIR")
        
        data.get_data_sjvair()
        
    elif datatypes.upper() == 'PURPLEAIR':
        
        data = GetPurpleAirData(
            sensor_list=["155683", "104106"],
            start_date=datetime.datetime(2023, 11, 16,4,30,0),
            end_date=datetime.datetime(2023, 11, 16,7,0,0),
            headers={'X-API-Key': "5E56E968-4C34-11EE-A77F-42010A800009"},
            fields=["pm2.5_alt","pm2.5_atm","pressure","humidity"]
        )
        
        data.get_data_purpleair()
        
    else:
        print("Please Choose Between PurpleAir or SjvAir")

Finish Download purpleair_folder/sensor_155683_data.csv
Finish Download purpleair_folder/sensor_104106_data.csv
